# JOB OFFER'S DATA (part 3: merging and cleaning)

In [1]:
# Load nessesary libraries.
import pandas as pd
import re
from random import randint
from datetime import datetime, timedelta

In [2]:
# Define global variables.
label_value = ['registered_nurse', 'electrician', 'data_analyst']
csv_file_name = ['data_jobads_rn.csv', 'data_jobads_e.csv', 'data_jobads_da.csv']
data_frame = ['df1', 'df2', 'df3']
to_remove = ['salary', 'schedule', 'benefits', 'location', 'job type', 'office']
date_of_download = "January 10, 2024"
before_30_days = 'before December 11, 2023'

In [3]:
# Define a custom function to import a CSV file, add a label column, and return the DataFrame.
def import_and_label(csv, data_f, value):
    
    # Read the CSV file into a DataFrame.
    data_f = pd.read_csv(csv, index_col=None)
    
    # Add a new column 'label' with the specified value.
    data_f['label'] = value
    
    # eturn the modified DataFrame
    return data_f

In [4]:
# Call the import_and_label function for the each CSV file.
df1 = import_and_label(csv_file_name[0], data_frame[0], label_value[0])
df2 = import_and_label(csv_file_name[1], data_frame[1], label_value[1])
df3 = import_and_label(csv_file_name[2], data_frame[2], label_value[2])

# Concatenate the three DataFrames along the rows (axis=0) and reset the index.
df = pd.concat([df1, df2, df3], ignore_index=True)

# Check the DataFrame.
print(df.shape)
df.head(2)

(1034, 6)


,title,id,link,date,job_description,label
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Just posted,Silver Stream Healthcare Group offer great emp...,registered_nurse
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Hiring ongoing,Create A Better Future For Yourself !!\nRecrui...,registered_nurse


**'date' column**

In [5]:
# Define a custom function to replace 'Just posted' or 'Today' with 'January 10, 2024'.
def replace_just_posted_today(date):
    global date_of_download
    return date_of_download if 'Just posted' in str(date) or 'Today' in str(date) else date

# Define a custom function extract and replace with relevant information.
def remove_elements_with_colon(date):
    date = replace_just_posted_today(date)
    
    if date and ('day' in str(date) or 'days' in str(date)):
        days_ago = int(re.search(r'(\d+) (?:day|days) ago', str(date)).group(1))
        new_date = datetime.strptime('January 10, 2024', '%B %d, %Y') - timedelta(days=days_ago)
        return new_date.strftime('%B %d, %Y')
    else:
        return date

In [6]:
# Apply changes.
df['date'] = df['date'].apply(replace_just_posted_today)
df['date'] = df['date'].replace(['not available', 'Hiring ongoing', None], 'unknown')
df['date'] = df['date'].replace(['Posted 30+ days ago'], before_30_days)
df['date'] = df['date'].apply(remove_elements_with_colon)

# Assuming df is your DataFrame and 'date' is the column
date_counts = df['date'].value_counts()

# Print the counts for each unique value in the 'date' column
print(date_counts)

before December 11, 2023    511
unknown                      84
January 08, 2024             71
December 20, 2023            46
January 05, 2024             45
December 22, 2023            37
January 10, 2024             30
January 03, 2024             30
January 09, 2024             28
January 04, 2024             25
January 06, 2024             18
December 13, 2023            14
December 19, 2023            12
December 14, 2023            11
December 12, 2023            10
December 21, 2023            10
December 23, 2023             8
December 16, 2023             7
December 30, 2023             6
January 02, 2024              6
December 15, 2023             5
January 07, 2024              5
December 29, 2023             3
December 18, 2023             2
January 01, 2024              2
December 28, 2023             2
December 26, 2023             2
December 31, 2023             1
December 11, 2023             1
December 24, 2023             1
December 27, 2023             1
Name: da

**'job_description' column**

In [7]:
# Define the costum function to find and remove unnecessary or private information.
def remove_elements_with_colon(column_value, remove_value):
        
    # Convert the value to lowercase.
    column_value = column_value.lower()
    
    # Split each value of the column into a list using '\n' as a separator.
    elements = column_value.split('\n')
    
    # Find the index of the element containing given value.
    index_of_element = next((i for i, elements in enumerate(elements) if remove_value in elements and ':' in elements), None)
    
    # Check if the first conditions are present in the text.
    if index_of_element is not None:
        next_colon_index = next((j for j in range(index_of_element + 1, len(elements)) if ':' in elements[j]), None)
        
        # Add an extra condition to check the second conditions are present in the text.
        if next_colon_index is not None:
            del elements[index_of_element:next_colon_index]
            return '\n'.join(elements)
        else:
            return column_value
        
    else:
        return column_value

In [8]:
# Apply the function to the column.
for val_rem in to_remove:
    df['job_description'] = df['job_description'].apply(lambda x: remove_elements_with_colon(x, val_rem))

# Print the modified DataFrame.
print(df.iat[randint(0, 1034), 4])
print('---------')
print(df.iat[randint(0, 1034), 4])

clarity are currently recruiting for a staff nurse - theatre, pacu & anaesthetics for a prestigious hospital in south dublin

the ideal candidate must be/have:
be active on the live nmbi register -
decision letters will not be accepted
bls & acls certifications
minimum of 1 year post registration acute experience - preferably in anaesthetics
management course completion desirable
the ability to demonstrate evidence of recent and relevant experience in the specialised area and in an acute hospital setting
have a high level of flexibility as required by the hospital
for further information or to make an application, contact laura at: laura.mulchrone@clarityrecruitment.ie
is this role not for you? feel free to reach out today to discuss other nurse vacancies available.
---------
assistant director of nursing (permanent) dublin city north
may suit cnm3 looing to progress their career.
general nursing
purpose of the role
the post holder is primarily responsible for the provision of a high q